In [22]:
import os,json
from flask import Flask, redirect,session, url_for, request
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from spotipy.cache_handler import FlaskSessionCacheHandler

In [23]:
client_id = "258ba1badd9247c9b88999500e883d59"
client_secret = "d45f2362c5bd42718060bd8a58038f32"
redirect_uri = 'http://localhost:5000/callback'
scope= 'playlist-read-private' 
cache_handler = FlaskSessionCacheHandler(session)

In [24]:
sp = Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope,
    open_browser=True  # Opens a browser for authentication
))


In [25]:
sp

In [26]:
def get_playlists():
    """Fetch the current user's playlists."""
    playlists = []
    results = sp.current_user_playlists()

    while results:
        for item in results["items"]:
            playlists.append({
                "id": item["id"],
                "name": item["name"],
                "url": item["external_urls"]["spotify"]
            })
        results = sp.next(results) if results["next"] else None

    return playlists


In [27]:
# Fetch and save playlists
playlists_data = get_playlists()

with open("playlists.json", "w", encoding="utf-8") as f:
    json.dump({"items": playlists_data}, f, indent=4, ensure_ascii=False)

print("✅ Playlists saved to playlists.json")


✅ Playlists saved to playlists.json
